# Chapter 8. Load balancing

# Céline Comte

This Notebook computes numerical results for Chapter 8 of the Ph.D. thesis *Resource management in computer clusters: algorithm design and performance analysis* by Céline Comte. Specifically, the following piece of code computes the overall performance of the dynamic load balancing when there is a single job type. The obtained results are used for the paragraph "Number of tokens" in Section 8.3.1.

## Package imports and global variable definitions

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# uncomment this line if you prefer dynamic matplotlib plots
# %matplotlib notebook

# change the default figure size
pylab.rcParams['figure.figsize'] = (10.0, 6.0)
pylab.rcParams['legend.fontsize'] = 12

In [3]:
# manipulate dataframes
import pandas as pd

In [4]:
# global variables
ρρ = append( linspace(5., .01, 400, endpoint = False), linspace(.01, 0, 10, endpoint = False) )

## Compute performance

### State aggregation

The following aggregate formulas allow us to compute global performance metrics like the average blocking probability and the mean delay. We cannot use them to derive more detailed metrics like the probability that each server is idle or the mean number of jobs at each server. For this, we use the results obtained with the (slower) program ``exact-bipartite.c`` called by ``script.sh``.

**Stationary measure.**
We focus on the aggregate state $y = (y_1,\ldots,y_N)$ of the queue of available tokens.
Any stationary measure $\pi_N$ of this aggregate state is of the form
$$
\pi_N(y) = \frac1G \Lambda(y) \prod_{i=1}^N \left( \frac1{\mu_i} \right)^{\ell_i - y_i},
\quad \forall y \le \ell,
$$
where $G$ is a normalization constant,
and the function $\Lambda$ is defined recursively on
$\left\{y \in \mathbb{N}^N: y \le \ell\right\}$
by $\Lambda(0) = 1$ and
$$
\Lambda(y) = \frac1\nu \sum_{i: y_i > 0} \Lambda(y-e_i),
\quad \forall y \le \ell: y \neq 0.
$$
We focus on the stationary measure $\pi_N$ such that $\pi_N(0) = 1$, that is
$$
\pi_N(y) = \Lambda(y) \prod_{i=1}^N {\mu_i}^{y_i},
\quad \forall y \le \ell.
$$

If we expand the recursion of $\Lambda$, we obtain:
$$
\Lambda(y) = \binom{y_1 + \ldots + y_N}{y_1,\ldots,y_N} \left( \frac1\nu \right)^{y_1 + \ldots y_N},
\quad \forall y \le \ell.
$$
Doing the substitution in $\pi_N$ yields
$$
\pi_N(y) = \binom{y_1 + \ldots + y_N}{y_1,\ldots,y_N}
\prod_{i=1}^N \left( \frac{\mu_i}\nu \right)^{y_i},
\quad \forall y \le \ell.
$$

**State aggregation.**
The size of the state space of $y$ is exponential with the number of servers,
hence the previous formula can only be applied to small server pools.
We consider a second aggregate state that lives in a much smaller space.
Specifically, we focus on the total number $t$ of available tokens, regardless of their class,
that is, $t = y_1 + \ldots + y_N$. Knowing the distribution of this second aggregate state is sufficient to compute performance metrics such as the average blocking probability or the mean number of jobs in the system.

With a slight abuse of notation, the stationary distribution of this new aggregate state is defined by
$$
\pi_N(t) = \sum_{\substack{y \le \ell: \\ y_1 + \ldots + y_N = t}} \pi_N(y),
\quad \forall t = 0,1,\ldots,\ell_1 + \ldots + \ell_N.
$$
Using the explicit expression of $\pi_N(y)$, we obtain,
for each $t = 1, \ldots, \ell_1 + \ldots + \ell_N$:
\begin{align*}
\pi_N(t)
&= \sum_{\substack{y \le \ell: \\ y_1 + \ldots + y_N = t}}
\binom{y_1 + \ldots + y_N}{y_1,\ldots,y_N}
\prod_{i=1}^N \left( \frac{\mu_i}\nu \right)^{y_i}, \\
&= \sum_{\substack{y \le \ell: \\ y_1 + \ldots + y_N = t}}
\frac{(y_1 + \ldots + y_N)!}{y_1! \times \ldots \times y_N!}
\prod_{i=1}^N \left( \frac{\mu_i}\nu \right)^{y_i}, \\
&= \sum_{\substack{y \le \ell: \\ y_1 + \ldots + y_N = t}}
\frac{t!}{y_1! \times \ldots \times y_N!}
\prod_{i=1}^N \left( \frac{\mu_i}\nu \right)^{y_i}, \\
&= t! \times
\sum_{\substack{s = 1, \ldots, \ell_1 + \ldots \ell_{N-1}, \\
y_N = 1\ldots,\ell_N: \\
s + y_N = t}}
\frac1{y_N!} \left( \frac{\mu_N}\nu \right)^{y_N}
\frac1{s!}
\sum_{\substack{(y_1, \ldots, y_{N-1}) \le (\ell_1, \ldots, \ell_{N-1}): \\
y_1 + \ldots + y_{N-1} = s}}
\frac{s!}{y_1! \times \ldots \times y_{N-1}!}
\prod_{i=1}^N \left( \frac{\mu_i}\nu \right)^{y_i}.
\end{align*}
We recognize the stationary measure $\pi_{N-1}$
of a server pool reduced to the first $N-1$ servers (with $\pi_{N-1}(0) = 1$).
In the end, we obtain
\begin{align*}
\pi_N(t)
&= t! \times
\sum_{\substack{s = 1, \ldots, \ell_1 + \ldots \ell_{N-1}, \\
y_N = 1\ldots,\ell_N: \\
s + y_N = t}}
\frac1{y_N!} \left( \frac{\mu_N}\nu \right)^{y_N}
\frac1{s!}
\pi_{N-1}(s)
\end{align*}

**Computation in practice.**
The last equality gives us an effective way of computing the stationary measure $\pi_N$
by progressively building $\pi_0$, $\pi_1$, $\ldots$, $\pi_{N-1}$, $\pi_N$,
where $\pi_0$ is defined on $\{0\}$ by $\pi_0(0) = 1$.

The function `dynamic` below leverages the functions of `numpy` library to speed up the computations.
The key is to observe that each component $\pi_N(t)$ of $\pi_N$
is equal (up to a multiplication by $\frac{t!}{s!}$) to
the sum of the coefficients of an antidiagonal
of the outer product of the vector $\pi_{N-1}$ and
$$
p_N = \left( \frac1{y_N!} \left( \frac{\mu_N}\nu \right)^{y_N}: y_N = 1, \ldots, \ell_N \right).
$$

### Functions

The function ``dynamic`` below computes the loss probability and the expected number of customers under the dynamic load-balancing algorithm introduced in Section 8.1, in the special case where there is a single job type. This corresponds to the scenario of Section 8.3.1. The state aggregation we use to speed up computations doesn't allow us to derive the activity probability of each computer.

In [5]:
def dynamic(I, ℓ, μ, ν, prefix=""):
    # parameters
    barℓ = zeros(I, dtype=int)
    barℓ[1:] = cumsum(ℓ[:-1])
    
    # initialization: π_0
    π = ones(1, dtype=float64)
    
    for i in range(I):
        # recursion:  derive π_{i+1} from π_i
        
        # compute p_{i+1}
        p = ones(ℓ[i] + 1, dtype=float64)
        p[1:] = cumprod(μ[i] / (ν * arange(1, ℓ[i]+1)), dtype=float64)
        
        # make the outer product of π_i with p_{i+1}
        π = outer(π, p[::-1])
        
        # multiply each coefficient by t! / s!
        for s in range(barℓ[i] + 1):
            quotient = ones(ℓ[i] + 1, dtype=float64)
            quotient[1:] = cumprod(s + arange(1, ℓ[i] + 1), dtype=float64)
            π[s] *= quotient[::-1]
        
        # sum over the (anti)diagonals
        π = [trace(π, n) for n in range(ℓ[i], -barℓ[i] - 1, -1)]
    
    # normalize π
    π /= sum(π)
    
    return π[0], inner(π, arange(sum(ℓ), -1, -1, dtype=float64))

The function ``create_dynamic_df`` calls the function ``dynamic`` and creates a dataframe (from ``pandas`` library) that stores the results:

In [6]:
def create_dynamic_df(I, ℓ, μ, ρρ, prefix=""):
    data = [dynamic(I, ℓ, μ, ρ * sum(μ)) for ρ in ρρ]
    
    return pd.DataFrame({
        'rho': ρρ,
        prefix + 'beta': [d[0] for d in data],
        prefix + 'eta': ρρ * [1. - d[0] for d in data],
        prefix + 'L': [d[1] for d in data],
        prefix + 'gamma': ρρ * sum(μ) * [(1. - d[0]) / d[1] for d in data],
    })

## A single job type

In [7]:
# parameters
I = 10
μ = ones(I); μ[I//2:] = 4.

### Impact of the number of tokens

In [8]:
# parameters
rg_tokens = [1, 2, 3, 6, 10]

In [9]:
# compute the analytical results
dynamic_tokens_df = pd.DataFrame({'rho': ρρ})

for nb_in_rg in rg_tokens:
    dynamic_tokens_df = pd.merge(
        dynamic_tokens_df,
        create_dynamic_df(I, nb_in_rg * ones(I, dtype=int), μ, ρρ, prefix=str(nb_in_rg)),
        on='rho',
    )

In [10]:
# save in csv
dynamic_tokens_df.to_csv("data/single-dynamic-exact-tokens.csv", index=False)